In [1]:
import os
import pickle
import tqdm
import functools
import multiprocessing
import numpy as np
import pandas as pd
import scipy.stats

import util_localization_psychophysics
import util_spkr_word_psychophysics


In [ ]:
"""
Run all sound localization experiments.

`list_regex_dir_model` is a list of model conditions
(different phase locking limits / cochlear models).

Each directory included must contain the model evaluation
output files for each psychophysical stimulus dataset (all
model predictions, output probabilities, and stimulus data).
"""

list_regex_dir_model = [
    "models/sound_localization/IHC3000_delayed_integration/arch??",
    "models/sound_localization/IHC3000/arch??",
    "models/sound_localization/IHC1000/arch??",
    "models/sound_localization/IHC0320/arch??",
    "models/sound_localization/IHC0050/arch??",
    "models/sound_localization/simplified_IHC3000_delayed_integration/arch??",
    "models/sound_localization/simplified_IHC3000/arch??",
    "models/sound_localization/simplified_IHC1000/arch??",
    "models/sound_localization/simplified_IHC0320/arch??",
    "models/sound_localization/simplified_IHC0050/arch??",
]

EXPERIMENT_DATAFRAMES = util_localization_psychophysics.run_localization_experiments(
    list_regex_dir_model,
    workers=60,
    dir_human_data='data/human/sound_localization',
    tag_ckpt='',
    func_label_to_azim_elev=util_localization_psychophysics.label_to_azim_elev,
    key_pred_prob='label_loc_int:probs_out',
    key_pred='label_loc_int:labels_pred',
    key_true='label_loc_int:labels_true',
    n_loc_classes=504,
)


In [ ]:
"""
Save sound localization experiment data to files.

A dictionary of all experiment results is saved to
a single pickle file. Individual experiment results
are also saved to separate CSV files.
"""

fn_data = 'data/model/sound_localization.pkl'
with open(fn_data, 'wb') as f:
    pickle.dump(EXPERIMENT_DATAFRAMES, f)
print(f"Wrote: {fn_data}")

for tag_expt in EXPERIMENT_DATAFRAMES.keys():
    fn_data_expt = fn_data.replace('.pkl', f'/{tag_expt}.csv')
    EXPERIMENT_DATAFRAMES[tag_expt].to_csv(fn_data_expt, index=False)
    print(f"Wrote: {fn_data_expt}")


In [ ]:
"""
Load all sound localization experiment data.
"""

fn_data = 'data/model/sound_localization.pkl'
with open(fn_data, 'rb') as f:
    EXPERIMENT_DATAFRAMES = pickle.load(f)
print(f"Loaded: {fn_data}")
for tag_expt in EXPERIMENT_DATAFRAMES.keys():
    print(f"|__ {tag_expt}")


In [ ]:
"""
Calculate human-model comparison metrics for
all sound localization experiments and save
output to a file.
"""

def rmse(x, y):
    return np.linalg.norm(x - y) / np.sqrt(x.shape[0])

dict_metric_function = {
    'pearsonr': scipy.stats.pearsonr,
    'rmse': rmse,
}

df_results = []
for tag_expt in tqdm.tqdm(EXPERIMENT_DATAFRAMES.keys()):
    df = EXPERIMENT_DATAFRAMES[tag_expt]
    df_results_tmp = util_localization_psychophysics.compare_localization_experiment(
        df,
        tag_expt=tag_expt,
        dict_metric_function=dict_metric_function,
        bootstrap_repeats=1000,
        workers=60,
    )
    if df_results_tmp is not None:
        df_results.append(df_results_tmp)
df_results = pd.concat(df_results)

fn_metrics = 'data/model/sound_localization_human_model_comparison_metrics.pkl'
df_results.to_pickle(fn_metrics)
print(f"Wrote: {fn_metrics}")


In [ ]:
"""
Run all word/voice recognition experiments.

`list_regex_dir_model` is a list of model conditions
(different phase locking limits / cochlear models).

Each directory included must contain the model evaluation
output files for each psychophysical stimulus dataset (all
model predictions and stimulus metadata).
"""

list_regex_dir_model = [
    "models/spkr_recognition/IHC3000/arch0_00??",
    "models/spkr_recognition/IHC1000/arch0_00??",
    "models/spkr_recognition/IHC0320/arch0_00??",
    "models/spkr_recognition/IHC0050/arch0_00??",
    "models/word_recognition/IHC3000/arch0_00??",
    "models/word_recognition/IHC1000/arch0_00??",
    "models/word_recognition/IHC0320/arch0_00??",
    "models/word_recognition/IHC0050/arch0_00??",
    "models/spkr_word_recognition/IHC3000/arch0_00??",
    "models/spkr_word_recognition/IHC1000/arch0_00??",
    "models/spkr_word_recognition/IHC0320/arch0_00??",
    "models/spkr_word_recognition/IHC0050/arch0_00??",
    "models/spkr_word_recognition/simplified_IHC3000/arch0_00??",
    "models/spkr_word_recognition/simplified_IHC1000/arch0_00??",
    "models/spkr_word_recognition/simplified_IHC0320/arch0_00??",
    "models/spkr_word_recognition/simplified_IHC0050/arch0_00??",
]

dict_basename_eval = {
    'pitch_altered': 'EVAL_pitch_altered_v00.json',
    'kell_like': 'EVAL_word_recognition_human_experiment_v00_foreground60dbspl.json',
    'speech_in_synthetic_textures': 'EVAL_word_recognition_speech_in_synthetic_textures.json',
    'hopkins_moore_2009': 'EVAL_hopkins_moore_2009.json',
    'kell_like_inharmonic': 'EVAL_word_recognition_human_experiment_v00_inharmonic_foreground60dbspl.json',
}

EXPERIMENT_DATAFRAMES = util_spkr_word_psychophysics.run_spkr_word_experiments(
    list_regex_dir_model,
    dict_basename_eval=dict_basename_eval,
)


In [ ]:
"""
Save word/voice recognition experiment data to files.

A dictionary of all experiment results is saved to
a single pickle file. Individual experiment results
are also saved to separate CSV files.
"""

fn_data = 'data/model/spkr_word_recognition.pkl'
with open(fn_data, 'wb') as f:
    pickle.dump(EXPERIMENT_DATAFRAMES, f)
print(f"Wrote: {fn_data}")

for tag_expt in EXPERIMENT_DATAFRAMES.keys():
    fn_data_expt = fn_data.replace('.pkl', f'/{tag_expt}.csv')
    EXPERIMENT_DATAFRAMES[tag_expt].to_csv(fn_data_expt, index=False)
    print(f"Wrote: {fn_data_expt}")


In [3]:
"""
Load all word/voice recognition experiment data.
"""

fn_data = 'data/model/spkr_word_recognition.pkl'
with open(fn_data, 'rb') as f:
    EXPERIMENT_DATAFRAMES = pickle.load(f)
print(f"Loaded: {fn_data}")
for tag_expt in EXPERIMENT_DATAFRAMES.keys():
    print(f"|__ {tag_expt}")


Loaded: data/model/spkr_word_recognition.pkl
|__ kell_like_inharmonic
|__ kell_like
|__ speech_in_synthetic_textures
|__ pitch_altered
|__ hopkins_moore_2009_raw
|__ hopkins_moore_2009


In [4]:
"""
Calculate human-model comparison metrics for
all word/voice recognition experiments and save
output to a file.
"""

def rmse(x, y):
    return np.linalg.norm(x - y) / np.sqrt(x.shape[0])

dict_metric_function = {
    'pearsonr': scipy.stats.pearsonr,
    'rmse': rmse,
}

experiment_to_compfunc_map = {
    'kell_like_word': functools.partial(
        util_spkr_word_psychophysics.compare_word_recognition_kell_like,
        restrict_conditions=[0, 1, 2, 3]),
    'speech_in_synthetic_textures': functools.partial(
        util_spkr_word_psychophysics.compare_word_recognition_speech_in_synthetic_textures),
    'pitch_altered_spkr_word': functools.partial(
        util_spkr_word_psychophysics.compare_recognition_pitch_altered, key_task=['spkr', 'word']),
    'pitch_altered_word': functools.partial(
        util_spkr_word_psychophysics.compare_recognition_pitch_altered, key_task='word'),
    'pitch_altered_spkr': functools.partial(
        util_spkr_word_psychophysics.compare_recognition_pitch_altered, key_task='spkr'),
    'hopkins_moore_2009_word': functools.partial(
        util_spkr_word_psychophysics.compare_word_recognition_hopkins_moore_2009),
}

def func_to_parallelize(tag_expt):
    compfunc = experiment_to_compfunc_map[tag_expt]
    tag_expt_for_dataframe = tag_expt
    for suffix in ['_spkr', '_word', '_dip_listening']:
        tag_expt_for_dataframe = tag_expt_for_dataframe.replace(suffix, '')
    df = EXPERIMENT_DATAFRAMES[tag_expt_for_dataframe]
    df_results = None
    for metric_key, metric_function in dict_metric_function.items():
        df_results_tmp = compfunc(
            df,
            bootstrap_repeats=1000,
            metric_function=metric_function)
        df_results_tmp = df_results_tmp.rename(columns={
            'metric': metric_key,
            'bootstrap_list_metric': f'bootstrap_list_{metric_key}',
        })
        df_results_tmp['tag_expt'] = tag_expt
        if df_results is None:
            df_results = df_results_tmp
        else:
            df_results = df_results.merge(df_results_tmp, on=['tag_expt', 'tag_model'])
    return df_results


with multiprocessing.Pool(60) as pool:
    list_df_results = pool.map(func_to_parallelize, list(experiment_to_compfunc_map.keys()))
df_results = pd.concat(list_df_results)

fn_metrics = 'data/model/spkr_word_recognition_human_model_comparison_metrics.pkl'
df_results.to_pickle(fn_metrics)
print(f"Wrote: {fn_metrics}")


/om2/user/msaddler/.conda/envs/tf/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/om2/user/msaddler/.conda/envs/tf/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/om2/user/msaddler/.conda/envs/tf/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/om2/user/msaddler/.conda/envs/tf/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/om2/user/msaddler/.conda/envs/tf/lib/python3.11/sit

Wrote: data/model/spkr_word_recognition_human_model_comparison_metrics.pkl
